In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras

In [1]:
frame=pd.read_csv('bbc-text.csv')
frame

NameError: name 'pd' is not defined

In [4]:
frame['category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [5]:
data=frame.sample(frac=1)
data

,category,text
1867,sport,federer breezes into semi-finals roger federer...
1044,tech,domain system scam fear a system to make it ea...
1970,sport,balco case trial date pushed back the trial da...
936,sport,charvis set to lose fitness bid flanker colin ...
2179,entertainment,musicians to tackle us red tape musicians gro...
...,...,...
862,politics,leaders meet over turkish eu bid tony blair ha...
402,tech,warnings about junk mail deluge the amount of ...
770,sport,prop jones ready for hard graft adam jones say...
927,sport,wilkinson to lead england fly-half jonny wilki...


In [6]:
train_size=int(len(data)*.8)
print('train_size:',train_size)
print(len(data)-train_size)

train_size: 1780
445


In [7]:
def split_data(data,train_size):
    train=data[:train_size]
    test=data[train_size:]
    return train, test

In [8]:
train_cat, test_cat =split_data(data['category'], train_size)
train_text, test_text = split_data(data['text'], train_size)

In [9]:
test_text

280     levy takes whitbread novel prize orange prize ...
1322    edwards tips idowu for euro gold world outdoor...
1928    ukraine strikes turkmen gas deal ukraine has a...
850     gardener wins double in glasgow britain s jaso...
739     labour s election love-in peace and love have ...
                              ...                        
862     leaders meet over turkish eu bid tony blair ha...
402     warnings about junk mail deluge the amount of ...
770     prop jones ready for hard graft adam jones say...
927     wilkinson to lead england fly-half jonny wilki...
589     broadband fuels online change fast web access ...
Name: text, Length: 445, dtype: object

In [10]:
max_word=1000
tokenize=keras.preprocessing.text.Tokenizer(num_words=max_word, \
                                            char_level=False)

In [13]:
tokenize.fit_on_texts(train_text)
x_train=tokenize.texts_to_matrix(train_text)
x_test=tokenize.texts_to_matrix(test_text)
x_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [14]:
le=LabelEncoder()
y_train=le.fit_transform(train_cat)
encoded_y_train=keras.utils.to_categorical(y_train)
y_test=le.fit_transform(test_cat)
encoded_y_test=keras.utils.to_categorical(y_test)

In [15]:
batch_size=32
epochs=4
drop_ratio=0.5

In [16]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(max_word,)),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(212,activation='relu'),
    tf.keras.layers.Dense(5,activation='softmax')
])

In [17]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [18]:
history=model.fit(x_train,encoded_y_train, batch_size=batch_size, epochs=epochs,verbose=1,validation_split=0.1)

Train on 1602 samples, validate on 178 samples
Epoch 1/4
1602/1602 [==============================] - 28s 17ms/sample - loss: 0.4653 - accuracy: 0.8452 - val_loss: 0.1196 - val_accuracy: 0.9607
Epoch 2/4
1602/1602 [==============================] - 3s 2ms/sample - loss: 0.0359 - accuracy: 0.9881 - val_loss: 0.0960 - val_accuracy: 0.9719
Epoch 3/4
1602/1602 [==============================] - 3s 2ms/sample - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0888 - val_accuracy: 0.9663
Epoch 4/4
1602/1602 [==============================] - 4s 2ms/sample - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0942 - val_accuracy: 0.9663


In [19]:
result=model.evaluate(x_train,encoded_y_train,verbose=2)
result

1780/1 - 1s - loss: 0.1538 - accuracy: 0.9966


[0.010371879711119312, 0.99662924]

In [20]:
result1=model.evaluate(x_test,encoded_y_test,verbose=2)
result1

445/1 - 0s - loss: 0.2185 - accuracy: 0.9461


[0.18330602404776583, 0.9460674]

In [41]:
i=int(input('please enter a predected value:'))

please enter a predected value:200


In [42]:
prediction=model.predict(np.array([x_test[i]]))
prediction

array([[9.9998653e-01, 9.9828458e-06, 1.8861124e-07, 7.1318496e-08,
        3.2628921e-06]], dtype=float32)

In [43]:
text_labels = le.classes_
predection_label= text_labels[np.argmax(prediction)]
predection_label

'business'

In [44]:
print('Actual_lable: ',test_cat.iloc[i])
print('Predected_Lable: ',predection_label)

Actual_lable:  business
Predected_Lable:  business
